# GX Takes on Contract Classification Project 

## Task Description

Because the dataset was downloaded in week 43,
Use the entire training data set as training set, the week 38-43 dataset not uploaded as a testing set(a total of around 10k entries to verify). 
After updating the latest dataset, SQL query the dataset after PDate.

Also, can start by categorising is IT and Not-IT to reduce entries. 

Then only deal with is IT entries, and use a combination of rule based method and ML method to give category and subcategory.


## Featrues description
### __File 1: Master Contract Table (the main file we work on) Contains following fields: __

__CNID__: identifier.

ParentCN: we can at the moment not consider parent contract misclassification and focus on other aspects. – Discard;

__Current_Agency_Abbr__: possibly, as a shorter version, can be replaced through idagency.

SupplierName: important indicator. Need stemming?

PDate; ADate:don’t think matter. Discard

Status: don’t matter. Discard

SContract, EContract: merge to work out the period. Possibly construct new variable like daily rate to indicate labour hire?
ValueContract: value. Can determien SI service (if above 1 MM. can add rule?)

Oldstart: 
Oldvalue: 

__Title: or the description of the contract. Main analysis tool. The majority can be determined. Maybe focus on this field before moving on to the rest fields.__

AgencyRef: ?

austender_cat: spscs, can be indexed. – maybe together with agency can yield certain indications? – if someone were to consistently wrongly assign categories.

Procmethod: unsure.

AtmID: linked to procurement table. – another module to import

SonId: linked to panel table, can incorporate in python directly. 

AmendReason: unsure.

ATSupplierName: ?

SupplierPostcode: don’t know. Maybe indicates IT like in north sydney? Or use cluster to determine?

SupplierABN: maybe link to ABN search? Not important at the moment.

Category: Results
SubCategory: Results
Comment: Results

InternalComment: to standardise in the future, include.

GAPS ID: unsure;

AgencyTier: 

Original_Agency_Abbr: figure out diff. get rid of one either.

__OriginalSupplierName: figure out diff. get rid of one either.__
FinYear; FinQuarter; Month; Month Name: can discard.
ValueBand: could replace value? Correlated.

### __File 2: Discards table.__

Gives all the discards/non-ICT contracts. 3-4  times larger than ICT contract set.

### __File 3: Supplier table.__

use this table to match dirty name against clean name. Clean name is a smaller set can be used to assign Supplier categories.

### __File 4: Panel Table. Can give strict rule and change some obvious ones encountered.__

Category shows the most likely categories of the contracts. Only a few can indicate explitly the contract type, most indicate ICT/ Non-ICT.


## Stage 1: Data preprocessing

### Step1: Load files necessary

In [1]:
import sqlite3
import numpy as np
import pandas as pd
import nltk

In [2]:
contract = pd.read_csv('Contracts.csv') # main contract tabel

In [3]:
contract.head()

,CnId,IdAgency,idSupplier,Title,SonId,AtmID,IdCat
0,1045883,55,1204,To provide the service of Senior Unix Administ...,NaN,NaN,13
1,1119961,62,2409,Provision of IT Specialist Services by Specifi...,NaN,NaN,13
2,1120021,62,5073,Provision of IT Specialist Services by Specifi...,NaN,NaN,13
3,1124930,62,4141,Provision of IT Specialist Services by Specifi...,NaN,NaN,13
4,1125360,122,6094,Payment for services provided Plus additional ...,NaN,NaN,13


In [12]:
title = contract.Title
type(title)

pandas.core.series.Series

In [5]:
title.head()

0    To provide the service of Senior Unix Administ...
1    Provision of IT Specialist Services by Specifi...
2    Provision of IT Specialist Services by Specifi...
3    Provision of IT Specialist Services by Specifi...
4    Payment for services provided Plus additional ...
Name: Title, dtype: object

In [15]:
title[title.isnull()]

29330     NaN
45381     NaN
58453     NaN
59623     NaN
197870    NaN
260195    NaN
269311    NaN
282209    NaN
Name: Title, dtype: object

In [18]:
title.fillna(value="NODesc",inplace=True)

### Step2: Apply lemmatizer instead of stemmer due to existence of Technical terms.

In [10]:
# try stemming
from nltk.stem.snowball import EnglishStemmer
snowball_stemmer = EnglishStemmer()

In [24]:
stemmed = title.apply(snowball_stemmer.stem)
# remember to fillna otherwise error, float datatype.?
stemmed.head()
type(stemmed)

pandas.core.frame.DataFrame

The result shows that words are all lowered, which is not what we want.

In [22]:
wnl = nltk.WordNetLemmatizer()
lemmed = title.apply(wnl.lemmatize)
lemmed.head()

0    To provide the service of Senior Unix Administ...
1    Provision of IT Specialist Services by Specifi...
2    Provision of IT Specialist Services by Specifi...
3    Provision of IT Specialist Services by Specifi...
4    Payment for services provided Plus additional ...
Name: Title, dtype: object

In [27]:
type(lemmed)

pandas.core.series.Series